In [35]:
import urllib.request
import os
import tarfile

In [36]:
url="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath="data/aclImdb_v1.tar.gz"
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.isfile(filepath):
    result=urllib.request.urlretrieve(url,filepath)
    print('downloaded:',result)

In [37]:
if not os.path.exists("data/aclImdb"):
    tfile = tarfile.open("data/aclImdb_v1.tar.gz", 'r:gz')
    result=tfile.extractall('data/')

# 1. Import Library

In [38]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

# 數據準備

In [39]:
#讀取文件

In [40]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)

In [41]:
import os
def read_files(filetype):
    path = "data/aclImdb/"
    file_list=[]

    positive_path=path + filetype+"/pos/"
    for f in os.listdir(positive_path):
        file_list+=[positive_path+f]
    
    negative_path=path + filetype+"/neg/"
    for f in os.listdir(negative_path):
        file_list+=[negative_path+f]
        
    print('read',filetype, 'files:',len(file_list))
       
    all_labels = ([1] * 12500 + [0] * 12500) 
    
    all_texts  = []
    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
            
    return all_labels,all_texts

In [42]:
y_train,train_text=read_files("train")

read train files: 25000


In [43]:
y_test,test_text=read_files("test")

read test files: 25000


In [44]:
#查看正面評價的影評

In [45]:
train_text[0]

'well, this is an Ivan Reitman film. with the rare exception, Ivan likes to entertain. His films generally aren\'t "deep", but they are often entertaining enough. My Super Ex-Girlfriend surprised me in that i laughed more than i thought i would. Uma Thurman is just so grand, and i love her portrayals. I like Luke Wilson too, and Rainn Wilson was a straight hoot. Never taking itself seriously, the film is over the top and yet isn\'t very unique, nor does it go where no one has gone before.... it\'s a nice rent though and probably an OK date movie, especially if you have a headache and don\'t want to strain your brain. It\'s escapist fun and there\'s nothing wrong with that. When you strip away the "super-girl" stuff, you\'re left with a story about relationships, and relationships gone bad. It\'s a boy meets girl, boy leaves girl thing. And in the end, the characters are looking for love. Not all of them take being "dumped" as well as they could....a slice of life with a twist.'

In [46]:
y_train[0]

1

In [47]:
#查看負面評價的影評

In [48]:
train_text[12499]

'Without "mental anachronism", this film which I would like to find in DVD offer an extraordinary diving in the vital and mental context of thought of the people before the "disenchantment of the world". That, there is thirty years, a director and a scenario writer could test one such empathy and such a romantic truth to do it of them masterpiece leaves me astounding. It would be necessary to be able to see and re-examine it film for better seizing than the temporal and cultural distance us to make lose of capacity to be included/understood, analyze and finally to accept of such or such example of "primitive thought". Because this thought maintaining almost impossible to feel in the secularized world however contain certain keys of our behavior, that only them future generations will be able to analyze with sufficient relevance. If somebody knows where I then to get a numerical copy or VHS to me or DVD\x85 thank you in advance.'

In [49]:
y_train[12499]

1

# 先读取所有文章建立字典，限制字典的数量为nb_words=2000

In [50]:
token = Tokenizer(num_words=2000)
token.fit_on_texts(train_text)

In [51]:
#Tokenizer屬性

In [52]:
#fit_on_texts 讀取多少文章

In [53]:
print(token.document_count)

25000


In [54]:
print(token.word_index)

{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'in': 7, 'it': 8, 'i': 9, 'this': 10, 'that': 11, 'was': 12, 'as': 13, 'for': 14, 'with': 15, 'movie': 16, 'but': 17, 'film': 18, 'on': 19, 'not': 20, 'you': 21, 'are': 22, 'his': 23, 'have': 24, 'be': 25, 'he': 26, 'one': 27, 'all': 28, 'at': 29, 'by': 30, 'an': 31, 'they': 32, 'who': 33, 'so': 34, 'from': 35, 'like': 36, 'her': 37, 'or': 38, 'just': 39, 'about': 40, "it's": 41, 'out': 42, 'has': 43, 'if': 44, 'some': 45, 'there': 46, 'what': 47, 'good': 48, 'more': 49, 'when': 50, 'very': 51, 'up': 52, 'no': 53, 'time': 54, 'she': 55, 'even': 56, 'my': 57, 'would': 58, 'which': 59, 'only': 60, 'story': 61, 'really': 62, 'see': 63, 'their': 64, 'had': 65, 'can': 66, 'were': 67, 'me': 68, 'well': 69, 'than': 70, 'we': 71, 'much': 72, 'been': 73, 'get': 74, 'bad': 75, 'will': 76, 'also': 77, 'do': 78, 'into': 79, 'people': 80, 'other': 81, 'first': 82, 'great': 83, 'because': 84, 'how': 85, 'him': 86, 'most': 87, "don't": 88, 'made

# 將每一篇文章的文字轉換一連串的數字，只有在字典中的文字會轉換為數字

In [55]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq  = token.texts_to_sequences(test_text)

In [56]:
print(train_text[0])

well, this is an Ivan Reitman film. with the rare exception, Ivan likes to entertain. His films generally aren't "deep", but they are often entertaining enough. My Super Ex-Girlfriend surprised me in that i laughed more than i thought i would. Uma Thurman is just so grand, and i love her portrayals. I like Luke Wilson too, and Rainn Wilson was a straight hoot. Never taking itself seriously, the film is over the top and yet isn't very unique, nor does it go where no one has gone before.... it's a nice rent though and probably an OK date movie, especially if you have a headache and don't want to strain your brain. It's escapist fun and there's nothing wrong with that. When you strip away the "super-girl" stuff, you're left with a story about relationships, and relationships gone bad. It's a boy meets girl, boy leaves girl thing. And in the end, the characters are looking for love. Not all of them take being "dumped" as well as they could....a slice of life with a twist.


In [57]:
print(x_train_seq[0])

[69, 10, 6, 31, 18, 15, 1, 1277, 1392, 1226, 5, 23, 104, 1220, 708, 929, 17, 32, 22, 395, 438, 191, 57, 1159, 1229, 975, 765, 68, 7, 11, 9, 1502, 49, 70, 9, 193, 9, 58, 6, 39, 34, 1754, 2, 9, 115, 37, 9, 36, 95, 2, 12, 3, 725, 111, 652, 406, 609, 1, 18, 6, 116, 1, 346, 2, 242, 214, 51, 949, 881, 123, 8, 136, 117, 53, 27, 43, 820, 154, 41, 3, 323, 849, 147, 2, 238, 31, 606, 1299, 16, 257, 44, 21, 24, 3, 2, 88, 177, 5, 125, 1221, 41, 249, 2, 221, 159, 351, 15, 11, 50, 21, 241, 1, 1159, 246, 534, 330, 313, 15, 3, 61, 40, 1514, 2, 1514, 820, 75, 41, 3, 426, 888, 246, 426, 885, 246, 151, 2, 7, 1, 126, 1, 101, 22, 263, 14, 115, 20, 28, 4, 94, 189, 108, 13, 69, 13, 32, 96, 3, 4, 109, 15, 3, 1004]


# 讓轉換後的數字長度相同

In [58]:
#文章內的文字，轉換為數字後，每一篇的文章所產生的數字長度都不同，因為之後需要進行類神經網絡的訓練，所以每一篇文章所產生的數字長度必須相同
#以下列程序代碼為例maxlen=100，所以每一篇文章轉換為數字都必須為100

In [59]:
x_train = pad_sequences(x_train_seq, maxlen=100)
x_test  = pad_sequences(x_test_seq,  maxlen=100)

In [60]:
#如果文章轉成數字大於0,pad_sequences處理後，會truncate前面的數字

In [61]:
print('before pad_sequences length=',len(x_train_seq[0]))
print(x_train_seq[0])

before pad_sequences length= 162
[69, 10, 6, 31, 18, 15, 1, 1277, 1392, 1226, 5, 23, 104, 1220, 708, 929, 17, 32, 22, 395, 438, 191, 57, 1159, 1229, 975, 765, 68, 7, 11, 9, 1502, 49, 70, 9, 193, 9, 58, 6, 39, 34, 1754, 2, 9, 115, 37, 9, 36, 95, 2, 12, 3, 725, 111, 652, 406, 609, 1, 18, 6, 116, 1, 346, 2, 242, 214, 51, 949, 881, 123, 8, 136, 117, 53, 27, 43, 820, 154, 41, 3, 323, 849, 147, 2, 238, 31, 606, 1299, 16, 257, 44, 21, 24, 3, 2, 88, 177, 5, 125, 1221, 41, 249, 2, 221, 159, 351, 15, 11, 50, 21, 241, 1, 1159, 246, 534, 330, 313, 15, 3, 61, 40, 1514, 2, 1514, 820, 75, 41, 3, 426, 888, 246, 426, 885, 246, 151, 2, 7, 1, 126, 1, 101, 22, 263, 14, 115, 20, 28, 4, 94, 189, 108, 13, 69, 13, 32, 96, 3, 4, 109, 15, 3, 1004]


In [62]:
print('after pad_sequences length=',len(x_train[0]))
print(x_train[0])

after pad_sequences length= 100
[ 346    2  242  214   51  949  881  123    8  136  117   53   27   43
  820  154   41    3  323  849  147    2  238   31  606 1299   16  257
   44   21   24    3    2   88  177    5  125 1221   41  249    2  221
  159  351   15   11   50   21  241    1 1159  246  534  330  313   15
    3   61   40 1514    2 1514  820   75   41    3  426  888  246  426
  885  246  151    2    7    1  126    1  101   22  263   14  115   20
   28    4   94  189  108   13   69   13   32   96    3    4  109   15
    3 1004]


In [63]:
#如果文章轉成數字不足100,pad_sequences處理後，前面會加上0

In [64]:
print('before pad_sequences length=',len(x_train_seq[1]))
print(x_train_seq[1])

before pad_sequences length= 36
[3, 51, 1723, 1086, 2, 69, 89, 18, 2, 1750, 293, 64, 551, 1, 624, 6, 1336, 1, 544, 135, 22, 476, 10, 6, 3, 18, 9, 76, 353, 145, 170, 2, 170, 27, 4, 57]


In [65]:
print('after pad_sequences length=',len(x_train[1]))
print(x_train[1])

after pad_sequences length= 100
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    3   51 1723 1086    2   69
   89   18    2 1750  293   64  551    1  624    6 1336    1  544  135
   22  476   10    6    3   18    9   76  353  145  170    2  170   27
    4   57]


# 數據預處理

In [66]:
token = Tokenizer(num_words=2000)
token.fit_on_texts(train_text)

In [67]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq  = token.texts_to_sequences(test_text)

In [68]:
x_train = pad_sequences(x_train_seq, maxlen=100)
x_test  = pad_sequences(x_test_seq,  maxlen=100)